In [1]:
#データをベクトル化する
#かつ、データについてshuffleする。
# -*- coding: utf-8 -*-
import math
import csv
import os
import json
import pickle
import random

shuffle_repeat = 100

def sentence_vectorize():
    
    #positive,negativeすべて合わせた文のインデックス用
    #後でシャッフル呼び出し時に用いる
    total_index = 0
    
    #positive,negative両方用
    all_sentence_vectorized = []
    #positive : sign =1, negative : sign = -1
    with open("./data/books/positive.review",mode = "r", encoding = "utf-8") as p:
        index = 0   #各文にインデックスを付ける。シャッフル呼び出し用
        positive = []
        for line in p:
            
            oneline = line.split(" ") #まだ　" " でsplitしただけ
            one_sentence = [0 for _ in range(200000)]  #これから一文をベクトル化する
            
            #一文内の各id:countに対する処理
            for one_id_count_set in oneline:
                id_,count_ = one_id_count_set.split(":")
                # 文末のid:count用
                count_ = count_.strip() 
                
                one_sentence[int(id_)] += int(count_) 

            #一文に対する処理　ここまで
            #one_sentenceを記録する、ただし、sign,indexをつける必要がある
            
            #sign, index用も込のリスト、すなわち
            #[ sign ,  index,  [ 200000次元のリスト ]]　とする。
            
            
            
            if index % 500 == 0:
                print(str(index) + " sentence is vectorised")
            
            positive.append(one_sentence)
                
            #各sentenceに固有
            index +=1
        # shuffle  
        for i in range(shuffle_repeat):
            positive = random.sample(positive,len(positive))
        with open("./positive.review.vector","wb") as pp:
            pickle.dump(positive,pp)
        
        #negative_dataの各文章に対するインデックス開始は1から
        #total_indexは全センテンス用
        total_index += index
        all_sentence_vectorized.append(positive)

    with open("./data/books/negative.review",mode = "r", encoding = "utf-8") as n:
        index = 0   #各文にインデックスを付ける。シャッフル呼び出し用
        negative = []
        for line in n:
            
            oneline = line.split(" ") #まだ　" " でsplitしただけ
            one_sentence = [0 for _ in range(200000)]  #これから一文をベクトル化する
            
            #一文内の各id:countに対する処理
            for one_id_count_set in oneline:
                id_,count_ = one_id_count_set.split(":")
                # 文末のid:count用
                count_ = count_.strip() 
                
                one_sentence[int(id_)] += int(count_) 
            #一文に対する処理　ここまで
            #one_sentenceを記録する、ただし、sign,indexをつける必要がある
            
            #sign, index用も込のリスト、すなわち
            #[ sign ,  index,  [ 200000次元のリスト ]]　とする。
            
            #sentence_to_be_recorded = [index,-1,one_sentence]
            
            if index % 500 == 0:
                print(str(index) + " sentence is vectorised")
            
            negative.append(one_sentence)
                
            #各sentenceに固有
            index +=1
            total_index +=1 
        for i in range(shuffle_repeat):
            negative = random.sample(negative,len(negative))
    all_sentence_vectorized.append(negative)
    
   
    with open("./negative.review.vector","wb") as nn:
            pickle.dump(negative,nn) 
            
        
    with open("./all_sentence_vectorised.vector","wb") as f:
        pickle.dump(all_sentence_vectorized,f)
        
    print(str(total_index) + " sentence is vectorised")
    
sentence_vectorize()

0 sentence is vectorised
500 sentence is vectorised
0 sentence is vectorised
500 sentence is vectorised
2000 sentence is vectorised


In [2]:
#split train=0.8, dev = 0.1, test = 0.1
#bias is needed

import random
import pickle

def data_spliter():
    p_train = []
    p_dev = []
    p_test = []
    
    n_train = []
    n_dev = []
    n_test = []
    
    with open("positive.review.vector","rb") as p:
        p_list = pickle.load(p)

        for i in range(len( p_list)):
            if i <800:
                p_train.append(p_list[i])
            elif i >= 800 and i <= 900:
                p_dev.append(p_list[i])
            else:
                p_test.append(p_list[i])
                
    with open("negative.review.vector","rb") as n:
        n_list = pickle.load(n)

        for i in range(len(n_list)):
            if i <800:
                n_train.append(n_list[i])
            elif i >= 800 and i < 900:
                n_dev.append(n_list[i])
            else:
                n_test.append(n_list[i])    
        
    with open("./train_dev_test/p_train","wb") as f:
        pickle.dump(p_train,f)

    with open("./train_dev_test/p_dev","wb") as f:
        pickle.dump(p_dev,f)

    with open("./train_dev_test/p_test","wb") as f:
        pickle.dump(p_test,f)
        
    with open("./train_dev_test/n_train","wb") as f:
        pickle.dump(n_train,f)

    with open("./train_dev_test/n_dev","wb") as f:
        pickle.dump(n_dev,f)

    with open("./train_dev_test/n_test","wb") as f:
        pickle.dump(n_test,f)
        
data_spliter()

In [3]:
#function

import math
def sigmoid(xx):
    if xx > 0:
        return (1 / (1 + math.exp(-xx)))
    else:
        return (1 - 1 / (1 + math.exp(xx)))
        

def v_dot(list_x,list_y): #内積
    sums = 0
    for i in range(len(list_x)):
        sums+= list_x[i] * list_y[i]
    return sums

def v_plus(list_x,list_y):
    result = [0 for _ in range(len(list_x))]
    for i in range(len(list_x)):
        result[i] = list_x[i] + list_y[i]
    return result    

def v_minus(list_x,list_y):
    result = [0 for _ in range(len(list_x))]
    for i in range(len(list_x)):
        result[i] = list_x[i] - list_y[i]
    return result    

def v_a_fold(a,list_x): #vectorの定数倍
    for i in list_x:
        i = a * i
    return list_x


In [4]:
import numpy as np
np.random.choice([i for i in range(1000)],20)

array([965, 521, 923, 370, 673, 279, 902, 644, 156, 946, 854, 209, 203,
       207, 441, 157, 830, 321, 257, 330])

In [5]:
import random
import math
import numpy as np

def load_train(batch_size):
    p_ret = []
    n_ret = []
    p_index = np.random.choice([i for i in range(800)],batch_size)
    n_index = np.random.choice([i for i in range(800)],batch_size)
    with open("./train_dev_test/p_train","rb") as p:
        with open("./train_dev_test/n_train","rb") as n:
            p_train = pickle.load(p)
            n_train = pickle.load(n)
            
            for _ in p_index:
                p_ret.append(p_train[_])
                
            for _ in n_index:
                n_ret.append(n_train[_])
    return p_ret, n_ret



def training(w,b,p_batch,n_batch,learning_rate,mini_batch_size):
    ips = 10 ** (-7)
    
    loss = 0
    one_iter = np.array( [0 for i in range(200000)])
    one_b = 0
    
    # 1 epochの中でp_batch(size = load_batch), n_batch(size = load_batch) 取ってきて
    #　その中からランダムで　mini_batch　個のデータで　1epoch回す。
    
    """
    for batch in p_train_batchs :
        y = 1
        one_iter = v_plus(one_iter ,  v_a_fold(((-1) * (y) * sigmoid((-1) * y * np.dot(w,batch))),batch) )
        one_b = one_b + (-1) * y * 1 * sigmoid((-1) * y * np.dot(w,batch))
        loss += (-1) * math.log(ips + sigmoid(y * np.dot(w,one_iter) + b))    
        #w = np.add(w ,(-1)* (learning_rate / len(batch)) * one_iter)
        
    w = np.add(w, v_a_fold((-1)* (learning_rate / len(p_train_batchs)) , one_iter) )
    #b = np.add( b,  ((-1)* (learning_rate / len(batch)) )* one_b)
    bias = bias - ((-1)* (learning_rate / len(p_train_batchs)) )* one_b      
    print("Loss: " + str(loss))
    
    loss = 0
    one_iter = np.array( [0 for i in range(200000)])
    one_b = 0
    for batch in n_train_batchs :
        y = -1
        one_iter = v_plus(one_iter ,  v_a_fold(((-1) * (y) * sigmoid((-1) * y * np.dot(w,batch))),batch) )
        one_b = one_b + (-1) * y * 1 * sigmoid((-1) * y * np.dot(w,batch))
        loss += (-1) * math.log(ips + sigmoid(y * np.dot(w,one_iter) + b))    
    #w = np.add(w ,(-1)* (learning_rate / len(batch)) * one_iter) 
    w = np.add(w, v_a_fold((-1)* (learning_rate / len(n_train_batchs)) , one_iter) )
    #b = np.add( b,  ((-1)* (learning_rate / len(batch)) )* one_b)
    bias = bias - ((-1)* (learning_rate / len(n_train_batchs)) )* one_b      
    print("Loss: " + str(loss))    
    
    
    """
    #print("p_batch"+str(p_batch))

    indexed_sentence = []
    for sentence in p_batch:
        indexed_sentence.append([1,sentence])
        
    for sentence in n_batch:
        indexed_sentence.append([-1,sentence])
        
    batch_to_be_used_index = np.random.choice(len(indexed_sentence),mini_batch_size)
    batch_to_be_used = []
    for _ in batch_to_be_used_index:
        batch_to_be_used.append(indexed_sentence[_])
    
    #print(batch_to_be_used)
    Loss = 0
    delL_devide_delw = np.array([0 for _ in range(200000)])
    delL_devide_delw_for_b = 0
    
    """
    delL_devide_delw = -y * sigmoid(-y *( np.dot(w,x) + b)) * x
    delL_devide_delw_for_b  = (-y) * 1 * sigmoid(-y *( np.dot(w,x) + b))
    
    """
    #w,b  : fixed
    for pn_sentence in batch_to_be_used:
        y = pn_sentence[0]
        #print(y)
        x = pn_sentence[1]
        x = np.array(x)
        #print(x,len(w))
        delL_devide_delw = np.array(delL_devide_delw) + (-1) * y * sigmoid((-1) * (y) * (np.dot(w,x)) )* x
        delL_devide_delw_for_b += (-1) * y * sigmoid((-1) * (y) * (np.dot(w,x)) )* 1 # x内に成分1がappendされて,wxを考える
        Loss += (-1) * math.log(ips +sigmoid(y * np.dot(w,x)))
    #learning
    w = w - ( learning_rate / len(batch_to_be_used) ) * delL_devide_delw
    b  =  b -  ( learning_rate / len(batch_to_be_used) ) * delL_devide_delw_for_b
    Loss = Loss / len(batch_to_be_used)
    
    return w,b,Loss

In [6]:
def load_dev():
    p_dev = []
    n_dev = []
    with open("./train_dev_test/p_dev","rb") as p:
        with open("./train_dev_test/n_dev","rb") as n:
            p_dev = pickle.load(p)
            n_dev = pickle.load(n)
    return p_dev, n_dev
    
def accuracy_eval(w,b):
    p_dev,n_dev = load_dev()
    correct = 0
    all_count = 0
    
    for _ in p_dev:
        all_count += 1
        y_ = 1
        _ = np.array(_)
        if np.dot(w,_) + b >=0:
            correct += 1
    for _ in n_dev:
        all_count += 1
        y_ = -1
        _ = np.array(_)
        if ((np.dot(w,_) + b)) <0:
            correct += 1    
    accuracy = correct * 100 / all_count
    return accuracy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epoch_times =400

load_batch = 100
mini_batch_size = 5
learning_rate = 1

w_tr = np.random.rand(200000)
b_tr = np.random.rand()

Loss_list = []
acc_list = []
for _ in range(epoch_times):
    p_trainbatch, n_trainbatch = load_train(load_batch)
    w_, b_, Loss = training(w_tr,b_tr, p_trainbatch,n_trainbatch,learning_rate,mini_batch_size)
    print("epoch" + str(_ + 1) + " end")
    print("Loss: "+str(Loss))
    accuracy = accuracy_eval( w_, b_ )
    print("accuracy: "+str(accuracy))
    w_tr = w_
    b_tr = b_
    Loss_list.append(Loss)
    acc_list.append(accuracy)
    
with open("./w_record_mb5_lr1","wb") as f:
    pickle.dump(w_tr,f)
    
with open("./b_record_mb5_lr1","wb") as f:
    pickle.dump(b_tr,f)
    
with open("./record/acc_record_mb5_lr1","wb") as f:
    pickle.dump(acc_list,f)
    
with open("./record/loss_record_mb5_lr1","wb") as f:
    pickle.dump(Loss_list,f)
    
plt.plot(Loss_list)
plt.show()#Loss
print()
plt.figure()
plt.plot(accuracy_list)
plt.show()
#accuracy

epoch1 end
Loss: 2.4285127753263724
accuracy: 50.24875621890547
epoch2 end
Loss: 4.413280672131117
accuracy: 50.24875621890547
epoch3 end
Loss: 2.708014429409851
accuracy: 50.24875621890547
epoch4 end
Loss: 3.3577462761204537
accuracy: 50.24875621890547
epoch5 end
Loss: 3.334225065121013
accuracy: 48.756218905472636
epoch6 end
Loss: 3.2339428428136285
accuracy: 48.756218905472636
epoch7 end
Loss: 1.5279791972772587
accuracy: 48.25870646766169
epoch8 end
Loss: 2.1804298172417313
accuracy: 49.25373134328358
epoch9 end
Loss: 11.661960074608075
accuracy: 46.766169154228855
epoch10 end
Loss: 2.1339382675378973
accuracy: 46.26865671641791
epoch11 end
Loss: 6.206453159884203
accuracy: 46.26865671641791
epoch12 end
Loss: 3.330428795222339
accuracy: 45.27363184079602
epoch13 end
Loss: 11.724050408822148
accuracy: 47.76119402985075
epoch14 end
Loss: 5.420869948143198
accuracy: 51.243781094527364
epoch15 end
Loss: 3.444220755381393
accuracy: 50.74626865671642
epoch16 end
Loss: 4.620538911303326
a

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epoch_times =400

load_batch = 100
mini_batch_size = 5
learning_rate = 0.1

w_tr = np.random.rand(200000)
b_tr = np.random.rand()

Loss_list = []
acc_list = []
for _ in range(epoch_times):
    p_trainbatch, n_trainbatch = load_train(load_batch)
    w_, b_, Loss = training(w_tr,b_tr, p_trainbatch,n_trainbatch,learning_rate,mini_batch_size)
    print("epoch" + str(_ + 1) + " end")
    print("Loss: "+str(Loss))
    accuracy = accuracy_eval( w_, b_ )
    print("accuracy: "+str(accuracy))
    w_tr = w_
    b_tr = b_
    Loss_list.append(Loss)
    acc_list.append(accuracy)
    
with open("./w_record_mb5_lr0p1","wb") as f:
    pickle.dump(w_tr,f)
    
with open("./b_record_mb5_lr0p1","wb") as f:
    pickle.dump(b_tr,f)
    
with open("./record/acc_record_mb5_lr0p1","wb") as f:
    pickle.dump(acc_list,f)
    
with open("./record/loss_record_mb5_lr0p1","wb") as f:
    pickle.dump(Loss_list,f)
    
plt.plot(Loss_list)
plt.show()#Loss
print()
plt.figure()
plt.plot(accuracy_list)
plt.show()
#accuracy